# 第6章 深層学習による3次元点群処理

## 6.1 深層学習の基礎

In [1]:
import torch

#入力は128次元のベクトル
input_data = torch.rand([128])
print(input_data.shape)

torch.Size([128])


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as  F

#入力128次元、出力256次元
linear = nn.Linear(128, 256)
input_data = torch.zeros((128))

x = input_data
#線形変換
x = linear(x)
#活性化関数はReLU
x = F.relu(x)
print(x.shape)

torch.Size([256])


In [5]:
#MLP : 多層パーセプトロンの実装
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear1 = nn.Linear(128, 256)
        self.linear2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x
    
net = MLP()
y = net(input_data)
print(y.shape)

torch.Size([10])


In [7]:
nrt = nn.Sequential(nn.Linear(128, 256), nn.ReLU(), nn.Linear(256, 10))
y = net(input_data)
print(y.shape)

torch.Size([10])


In [8]:
#ミニバッチの要素数×入力ベクトルの次元
input_data = torch.rand([32, 128])
y = net(input_data)
print(y.shape)

torch.Size([32, 10])


### Dockerができなかった

## 6.2 Pytorch Geometricによる3次元点群の扱い
- Pytorch Geometric 
    - 点群やメッシュをグラフとして出力する
        - 頂点の位置情報 : position
        - 各頂点の特徴量 : x
        - 各頂点の法線 : normal
        - 辺の張られ方 : edge_index
        - 各辺の特徴量 : edge_attr
        - 面の張られ方 : face
        - グラフ全体の特徴量・物体カテゴリ : y

In [17]:
#Pytorch GeimetricのModelNetデータセット機能を使う。ダウンロードして、前処理してデータを保存
from pathlib import Path
from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T

#一時的に保存するディレクトリを指定
current_path = Path.cwd()
dataset_dir = current_path / "modelnet10"

#点群に適用する前処理を指定
pre_transform = T.Compose([
    T.SamplePoints(1024, remove_faces=True, include_normals=True),
    T.NormalizeScale(),
])

train_dataset = ModelNet(dataset_dir, name="10", train=True, transform=None,
                        pre_transform=pre_transform, pre_filter=None)
test_dataset = ModelNet(dataset_dir, name="10", train=False, transform=None,
                       pre_transform=pre_transform, pre_filter=None)

Extracting /Users/sotaaraki/3dpcp_book_codes/notebook/modelnet10/ModelNet10.zip
Processing...
Done!


In [18]:
print("train_dataset len : ", len(train_dataset))
print(train_dataset[0])

train_dataset len :  3991
Data(pos=[1024, 3], y=[1], normal=[1024, 3])


In [19]:
print(train_dataset[0].pos.shape)
print(train_dataset[0].pos)

torch.Size([1024, 3])
tensor([[-0.3016, -0.0504, -0.1324],
        [-0.3738,  0.0773, -0.0737],
        [-0.4516, -0.3701,  0.2578],
        ...,
        [ 0.2526,  0.1295, -0.2235],
        [ 0.4451,  0.4263,  0.1332],
        [ 0.1182, -0.3978, -0.2363]])


In [21]:
#ミニバッチとして学習するために、Batch型を準備する
#各データが1024点からなる点群を32個にまとめてミニバッチしたため、1024×32 = 32768点含まれている
from torch_geometric.data import DataLoader as DataLoader
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=False)
batch = next(iter(dataloader))
print(batch)

DataBatch(pos=[32768, 3], y=[32], normal=[32768, 3], batch=[32768], ptr=[33])


## 6.3 PointNet

PointNet
- 3次元点群
    - 順不同なデータ
        - symmetric Functionでうまく扱える
            - 入力データの順番が変わったとしても、出力が変わらないような関数を指す
- Shared MLPとMax-Poolingを組み合わせたネットワークを提案している
    - shared MLP
        - 各点について、チャンネル方向に同一のMLPを適用する手法
        - 画像だとPointwise Convolution, 1×1 Convolutionと呼ばれている構造
- Shared MLPの後は、Max-Poolingで全点の特徴量を集約する
    - これは多視点画像の処理において、各視点で計算した特徴量を全ての視点について集約するView-Poolingと同じアイデア
    - これはチャンネルごとに適用される
    - Maxを取ることで、点の順序によらない出力を得ることができる
        - 他にもMin, averageなどの方法がある
        - 順序によらないプーリング操作

In [41]:
from pathlib import Path

from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T

current_path = Path.cwd()
dataset_dir = current_path / "modelnet10"

pre_transform = T.Compose([
    T.SamplePoints(1024, remove_faces=True, include_normals=True),
    T.NormalizeScale(),
])

train_dataset = ModelNet(dataset_dir, name="10", train=True, transform=None, pre_transform=pre_transform, pre_filter=None)
test_dataset = ModelNet(dataset_dir, name="10", train=False, transform=None, pre_transform=pre_transform, pre_filter=None)

In [42]:
print("train_dataset len:", len(train_dataset))
print(train_dataset[0])

train_dataset len: 3991
Data(pos=[1024, 3], y=[1], normal=[1024, 3])


In [43]:
print(train_dataset[0].pos.shape)
print(train_dataset[0].pos)

torch.Size([1024, 3])
tensor([[-0.3016, -0.0504, -0.1324],
        [-0.3738,  0.0773, -0.0737],
        [-0.4516, -0.3701,  0.2578],
        ...,
        [ 0.2526,  0.1295, -0.2235],
        [ 0.4451,  0.4263,  0.1332],
        [ 0.1182, -0.3978, -0.2363]])


In [24]:
"""
num_graphs : batch内の点群数
torch.size : 点群数×出力特徴量の次元
"""
#Symmetric functionの実装
from torch_geometric.nn import global_max_pool
import torch.nn as nn

#Pytorchによるモデルを継承してる
class SymmFunction(nn.Module):
    def __init__(self):
        #Shaered MLPを定義している
        super(SymmFunction, self).__init__()
        self.shared_mlp = nn.Sequential(
        nn.Linear(3, 64), nn.BatchNorm1d(64), nn.ReLU(),
        nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
        nn.Linear(128, 512)
        )
        
    def forward(self, batch):
        #点ごとの特徴量の変換を表している
        x = self.shared_mlp(batch.pos)
        #みにバッチ中に含まれる点群単位でプーリングを行っている
        x = global_max_pool(x, batch.batch)
        return x
    
f = SymmFunction()
print(batch)
y = f(batch)
print(y.shape)

DataBatch(pos=[32768, 3], y=[32], normal=[32768, 3], batch=[32768], ptr=[33])
torch.Size([32, 512])


- T-Net
    - Spatial Transformer Network(STN)のアイデアを導入したモジュール
    - 入力された点群の回転を学習ベースで自動的に正規化することを目指して導入される
- 回転の正規化の流れ
    - Shaered MLPとMax-Poolingで特徴量を取り出し、さらにMLPで9次元ベクトルにする
    - この9次元ベクトルを3×3の回転行列だと考えて、入力点群全体にこの回転を適用する
        - 回転行列を作用させる
    - ネットワークの出力が回転行列そのものでなく、回転行列と単位行列の差分となるようにしてる
        - これによって、学習する要素は単位行列からの変化分のみをネットワークが出力すれば良い
        - ネットワークの負担が軽くなる

In [27]:
#回転行列の正規化
class InputTNet(nn.Module):
    def __init__(self):
        #Max-Poolingで1024次元のベクトルに変換
        super(InputTNet, self).__init__()
        self.input_mlp = nn.Sequential(
        nn.Linear(3, 64), nn.BatchNorm1d(64), nn.ReLU(),
        nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
        nn.Linear(128, 1024), nn.BatchNorm1d(1024), nn.ReLU()
        )
        #MLPで9次元まで変換する
        self.output_mlp = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(),
            nn.Linear(256, 9)
        )
    
    def forward(self, x, batch):
        x = self.input_mlp(x)
        x = global_max_pooling(x, batch)
        x = self.output_mlp(x)
        #9次元を3×3の回転行列にする
        x = x.view(-1, 3, 3)
        id_matrix = torch.eye(3).to(x.device).view(1, 3, 3).repeat(x.shape[0], 1, 1)
        x = id_matrix + x
        return x

- 特徴量に行列を作用させる
    - これは今の特徴量空間から変換先の特徴量空間への線型写像に対応する

In [29]:
class FeatureTNet(nn.Module):
    def __init__(self):
        super(FeatureTNet, self).__init__()
        self.input_mlp = nn.Sequential(
            nn.Linear(64, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, 1024), nn.BatchNorm1d(1024), nn.ReLU()
        )
        self.output_mlp = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(),
            nn.Linear(256, 64*64)
        )
        
    def forward(self, x, batch):
        x = self.input_mlp(x)
        x = global_max_pooling(x, batch)
        x = self.output_mlp(x)
        x = x.view(-1, 64, 64)
        id_matrix = torch.eye(64).to(x.device).view(1, 64, 64).repeat(x.shape[0], 1, 1)
        x = id_matrix + x
        return x

### PointNetによるクラス分類

In [38]:
class PointNetClassification(nn.Module):
    def __init__(self):
        super(PointNetClassification, self).__init__()
        #入力点群に対するT-Net
        self.inupt_tnet = InputTNet()
        #点ごとに特徴量を変換するShared MLPが二つ(mlp1, mlp2)
        self.mlp1 = nn.Sequential(
            nn.Linear(3, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 64), nn.BatchNorm1d(64), nn.ReLU(),
        )
        #特徴量空間でのT-Net
        self.feature_tnet = FeatureTNet()
        self.mlp2 = nn.Sequential(
            nn.Linear(64, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, 1024), nn.BatchNorm1d(1024), nn.ReLU()
        )
        self.mlp3 = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(p=0.3), 
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(p=0.3), 
            nn.Linear(256, 10)
        )
        
    def forward(self, batch_data):
        x = batch_data.pos
        
        #入力点群に対するT-Netによって、入力点群に対した回転行列を計算する
        input_transform = self.input_tnet(x, batch_data.batch)
        #回転行列は入力点群それぞれについて計算されており、結合されてTensorに格納されているため、それぞれ展開する
        #batch_data.batch : どの点はどの点群に属しているのか
        transform = input_transform[batch_data.batch, :, :]
        #bmm : batch matrix mutliplication
        x = torch.bmm(transform, x.view(-1 ,3, 1)).view(-1, 3)
        
        #正規化された点群をShared MLPに入力し、64次元の特徴量を得る
        x = self.mlp1(x)
        
        #これまで得られた、特徴量を特徴量空間でのT-Netに入力し、特徴量空間で作用させるための行列を計算する
        feature_transform = self.feature_tnet(x, batch_data.batch)
        transform = feature_transform[batch_data.batch, :, :]
        x = torch.bmm(transform, x.view(-1, 64))
        
        x = self.mlp2(x)
        x = global_max_pool(x, batch_data.batch)
        x = self.mlp3(x)
        
        return x, input_transform, feature_transform

In [39]:
#学習
import torch 
from torch.utils.tensorboard import SummaryWriter

num_epoch = 400
batch_size = 32

#CUDA環境でGPUができるか確認
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
#ここまでに実装したモデルを生成
model = PointNetClassification()
#デバイスに転送
model = model.to(device)

#optimizer関連の設定をしている
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=num_epoch // 4, gamma=0.5)

log_dir = current_path / "log_modelnet10_classification"
log_dir.mkdir(exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

train_dateloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

criteria = torch.nn.CrossEntropyLoss()

/Users/sotaaraki/opt/anaconda3/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [40]:
#学習ループ全体
from tqdm import tqdm

for epoch in tqdm(range(num_epoch)):
    model = model.train()
    
    losses = []
    for batch_data in tqdm(train_dataloader, total=len(train_dataloader)):
        batch_data = batch_data.to(device)
        this_batch_size = batch_data.batch.detach().max() + 1
        
        pred_y, _, feature_transform = model(batch_data)
        true_y = batch_data.y.detach()

        class_loss = criteria(pred_y, true_y)
        accuracy = float((pred_y.argmax(dim=1) == true_y).sum()) / float(this_batch_size)

        id_matrix = torch.eye(feature_transform.shape[1]).to(feature_transform.device).view(1, 64, 64).repeat(feature_transform.shape[0], 1, 1)
        transform_norm = torch.norm(torch.bmm(feature_transform, feature_transform.transpose(1, 2)) - id_matrix, dim=(1, 2))
        reg_loss = transform_norm.mean()

        loss = class_loss + reg_loss * 0.001
        
        losses.append({
            "loss": loss.item(),
            "class_loss": class_loss.item(),
            "reg_loss": reg_loss.item(),
            "accuracy": accuracy,
            "seen": float(this_batch_size)})
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    if (epoch % 10 == 0):
        model_path = log_dir / f"model_{epoch:06}.pth"
        torch.save(model.state_dict(), model_path)
    
    loss = 0
    class_loss = 0
    reg_loss = 0
    accuracy = 0
    seen = 0
    for d in losses:
        seen = seen + d["seen"]
        loss = loss + d["loss"] * d["seen"]
        class_loss = class_loss + d["class_loss"] * d["seen"]
        reg_loss = reg_loss + d["reg_loss"] * d["seen"]
        accuracy = accuracy + d["accuracy"] * d["seen"]
    loss = loss / seen
    class_loss = class_loss / seen
    reg_loss = reg_loss / seen
    accuracy = accuracy / seen
    writer.add_scalar("train_epoch/loss", loss, epoch)
    writer.add_scalar("train_epoch/class_loss", class_loss, epoch)
    writer.add_scalar("train_epoch/reg_loss", reg_loss, epoch)
    writer.add_scalar("train_epoch/accuracy", accuracy, epoch)

    with torch.no_grad():
        model = model.eval()

        losses = []
        for batch_data in tqdm(test_dataloader, total=len(test_dataloader)):
            batch_data = batch_data.to(device)
            this_batch_size = batch_data.batch.detach().max() + 1

            pred_y, _, feature_transform = model(batch_data)
            true_y = batch_data.y.detach()

            class_loss = criteria(pred_y, true_y)
            accuracy =float((pred_y.argmax(dim=1) == true_y).sum()) / float(this_batch_size)

            id_matrix = torch.eye(feature_transform.shape[1]).to(feature_transform.device).view(1, 64, 64).repeat(feature_transform.shape[0], 1, 1)
            transform_norm = torch.norm(torch.bmm(feature_transform, feature_transform.transpose(1, 2)) - id_matrix, dim=(1, 2))
            reg_loss = transform_norm.mean()

            loss = class_loss + reg_loss * 0.001

            losses.append({
                "loss": loss.item(),
                "class_loss": class_loss.item(),
                "reg_loss": reg_loss.item(),
                "accuracy": accuracy,
                "seen": float(this_batch_size)})
            
        loss = 0
        class_loss = 0
        reg_loss = 0
        accuracy = 0
        seen = 0
        for d in losses:
            seen = seen + d["seen"]
            loss = loss + d["loss"] * d["seen"]
            class_loss = class_loss + d["class_loss"] * d["seen"]
            reg_loss = reg_loss + d["reg_loss"] * d["seen"]
            accuracy = accuracy + d["accuracy"] * d["seen"]
        loss = loss / seen
        class_loss = class_loss / seen
        reg_loss = reg_loss / seen
        accuracy = accuracy / seen
        writer.add_scalar("test_epoch/loss", loss, epoch)
        writer.add_scalar("test_epoch/class_loss", class_loss, epoch)
        writer.add_scalar("test_epoch/reg_loss", reg_loss, epoch)
        writer.add_scalar("test_epoch/accuracy", accuracy, epoch)

  0%|          | 0/400 [00:00<?, ?it/s]


NameError: name 'train_dataloader' is not defined

---

## 点群の畳み込み
- 